implementing a very naive attention mechanism 

In [2]:
import torch 

inputs = torch.tensor(
[[0.43, 0.15, 0.89], 
[0.55, 0.87, 0.66], 
[0.57, 0.85, 0.64],  
[0.22, 0.58, 0.33],  
[0.77, 0.25, 0.10],  
[0.05, 0.80, 0.55]])

In [3]:
attn_scores = torch.zeros(inputs.shape[0], inputs.shape[0])

'''for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i][j] = torch.dot(x_i, x_j)'''


attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [4]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights


tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [5]:
#context_vec = attn_weights @


context_vec = torch.zeros(inputs.shape)


'''for i, a_i in enumerate(attn_weights):
    for j, x_i in enumerate(inputs):
        context_vec[i] += a_i[j] * x_i'''

context_vec = attn_weights @ inputs
context_vec


tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

coding a better, trainable attention mechanism

In [12]:
x_2 = inputs[1]
d_in = x_2.shape[0]
d_out = x_2.shape[0] - 1

In [21]:
torch.manual_seed(123)

w_q = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_k = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
w_v = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [27]:
x_q = x_2 @ w_q
x_v = x_2 @ w_v
x_k = x_2 @ w_k
keys = inputs @ w_k
values = inputs @ w_v

In [26]:
print(x_k)
print(x_q)
print(x_v)

tensor([0.4433, 1.1419])
tensor([0.4306, 1.4551])
tensor([0.3951, 1.0037])


In [28]:
print(keys)
print(values)

tensor([[0.3669, 0.7646],
        [0.4433, 1.1419],
        [0.4361, 1.1156],
        [0.2408, 0.6706],
        [0.1827, 0.3292],
        [0.3275, 0.9642]])
tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])


In [33]:
attn_scores_2 = x_q @ keys.T

In [39]:
attn_scores_2_sm = torch.softmax(attn_scores_2, dim=-1)
attn_scores_2_sm

tensor([0.1401, 0.2507, 0.2406, 0.1157, 0.0687, 0.1842])

In [44]:
d_k = keys.shape[-1]
attn_scores_2_n = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1)
attn_scores_2_n

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

2